In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
import src.slurm_cluster as scluster
client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
dir1 ='./Figure3/ACCESS/'
ACCESS_forced = xr.open_dataset(dir1 + 'GSAT_ACCESS_Forced_anomalies_1850_2022.nc')
ACCESS_internal = xr.open_dataset(dir1 + 'GSAT_ACCESS_Internal_Variability_anomalies_1850_2022.nc')

In [ ]:
ACCESS_forced

In [ ]:
ACCESS_internal

### Calculate the trend in 10, 30, 60yr

In [ ]:
variable_name = ['10yr', '30yr', '60yr']
# define the function
def separate_data_into_intervals(data, start_year, end_year, time_interval):
    """
    This function is used to separate the data into different time intervals
    """
    # create a dictionary to store the data
    data_dict = {}
    for i in range(len(variable_name)):
        # calculate the start year and end year
        start_year = time_interval[variable_name[i]][0]
        end_year = time_interval[variable_name[i]][1]
        # select the data
        data_dict[variable_name[i]] = data.sel(year=slice(str(start_year), str(end_year)))
    return data_dict

In [ ]:
time_interval = {
    "10yr":(2013,2022),
    "30yr":(1993,2022),
    "60yr":(1963,2022)
}
start_year = 1950
end_year   = 2022

In [ ]:
ACCESS_forced_data_dict = separate_data_into_intervals(ACCESS_forced, start_year, end_year, time_interval)
ACCESS_unforced_data_dict = separate_data_into_intervals(ACCESS_internal, start_year, end_year, time_interval)

In [ ]:
ACCESS_forced_data_dict

In [ ]:
ACCESS_forced_data_dict['10yr'].tas[1,:,:,:]


In [ ]:
# Calculate the trend and p-value for each time interval of each realization
trend_dict = {}
pvalue_dict = {}

for i in range(len(variable_name)):
    data_var = ACCESS_forced_data_dict[variable_name[i]]['tas']
    print(data_var)
    
    slope, p_values = xr.apply_ufunc(
        func_mk,
        data_var,
        input_core_dims=[["year"]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float],
        dask_gufunc_kwargs={'allow_rechunk': True}
    )
    trend_dict[variable_name[i]] = slope
    pvalue_dict[variable_name[i]] = p_values

In [ ]:
trend_dict['10yr'][0,:,:]

In [ ]:
trend_dict['10yr'][1,:,:]

In [ ]:
trend_dict['30yr'][1,:,:]

In [ ]:
trend_annual_np = {}
pvalue_annual_np = {}

for i in range(len(variable_name)):
    trend_annual_np[variable_name[i]] = trend_dict[variable_name[i]].values
    pvalue_annual_np[variable_name[i]] = pvalue_dict[variable_name[i]].values
    
trend_annual_np['10yr']

In [ ]:
trend_annual_da = {}
pvalue_annual_da = {}

for interval, data in trend_annual_np.items():
    trend_annual_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": ACCESS_forced_data_dict[interval].run, 
    "lat": ACCESS_forced_data_dict[interval].lat, "lon": ACCESS_forced_data_dict[interval].lon})
for interval, data in pvalue_annual_np.items():
    pvalue_annual_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": ACCESS_forced_data_dict[interval].run, 
    "lat": ACCESS_forced_data_dict[interval].lat, "lon": ACCESS_forced_data_dict[interval].lon})

In [ ]:
# out put the forced trend and pvalue
dir_out ='./Figure3/ACCESS/trend/'

for interval, data in trend_annual_da.items():
    print(interval)
    print(data.values)
    data.to_netcdf(dir_out + 'ACCESS_forced_' + interval + '_trend.nc')
# for interval, data in pvalue_annual_da.items():
#     data.to_netcdf(dir_out + 'ACCESS_forced_' + interval + '_pvalue.nc')

### Calculated the unforced trend of each realization

In [ ]:
unforced_trend_dict = {}
unforced_pvalue_dict = {}

for i in range(len(variable_name)):
    data_var = ACCESS_unforced_data_dict[variable_name[i]]['tas']
    
    slope, p_values = xr.apply_ufunc(
        func_mk,
        data_var,
        input_core_dims=[["year"]],
        output_core_dims=[[], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float, float],
        dask_gufunc_kwargs={'allow_rechunk': True}
    )
    unforced_trend_dict[variable_name[i]] = slope
    unforced_pvalue_dict[variable_name[i]] = p_values

In [ ]:
unforced_trend_np = {}
unforced_pvalue_np = {}

for i in range(len(variable_name)):
    unforced_trend_np[variable_name[i]] = unforced_trend_dict[variable_name[i]].values
    unforced_pvalue_np[variable_name[i]] = unforced_pvalue_dict[variable_name[i]].values

unforced_trend_da = {}
unforced_pvalue_da = {}

for interval, data in unforced_trend_np.items():
    unforced_trend_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": ACCESS_unforced_data_dict[interval].run, 
    "lat": ACCESS_unforced_data_dict[interval].lat, "lon": ACCESS_unforced_data_dict[interval].lon})
for interval, data in unforced_pvalue_np.items():
    unforced_pvalue_da[interval] = xr.DataArray(data, dims=["run","lat", "lon"], coords={"run": ACCESS_unforced_data_dict[interval].run, 
    "lat": ACCESS_unforced_data_dict[interval].lat, "lon": ACCESS_unforced_data_dict[interval].lon})


In [ ]:
unforced_trend_da

In [ ]:
for interval, data in unforced_trend_da.items():
    data.to_netcdf(dir_out + 'ACCESS_unforced_' + interval + '_trend.nc')
for interval, data in unforced_pvalue_da.items():
    data.to_netcdf(dir_out + 'ACCESS_unforced_' + interval + '_pvalue.nc')

In [ ]:
unforced_trend_da

In [ ]:
ACCESS_unforced_10yr_trend = unforced_trend_da['10yr']
ACCESS_unforced_30yr_trend = unforced_trend_da['30yr']
ACCESS_unforced_60yr_trend = unforced_trend_da['60yr']

In [ ]:
# Input the observational forced and unforced trend wrt. ACCESS-CM6A-LR
dir_obs_ACCESS = '/work/mh0033/m301036/Land_surf_temp/analyses_1850_2100/Manuscript_visual_schematic/Disentangling_trend_analysis/Figure2/data/'

Obs_ACCESS_forced_10yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_forced_10yr_trend_ACCESS.nc')
Obs_ACCESS_forced_30yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_forced_30yr_trend_ACCESS.nc')
Obs_ACCESS_forced_60yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_forced_60yr_trend_ACCESS.nc')

Obs_ACCESS_unforced_10yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_internal_10yr_trend_ACCESS.nc')
Obs_ACCESS_unforced_30yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_internal_30yr_trend_ACCESS.nc')
Obs_ACCESS_unforced_60yr_trend = xr.open_dataset(dir_obs_ACCESS + 'HadCRUT5_annual_internal_60yr_trend_ACCESS.nc')

In [ ]:
import scipy.stats as stats

OBS_unforced_trend_pattern_correlation_10yr = []
for i in range(len(unforced_trend_da['10yr'].run)):
    OBS_unforced_trend_pattern_correlation_10yr.append(stats.pearsonr(ACCESS_unforced_10yr_trend.isel(run=i).values.flatten(), Obs_ACCESS_unforced_10yr_trend['tas'].values.flatten())[0])
    
OBS_unforced_trend_pattern_correlation_30yr = []
for i in range(len(unforced_trend_da['30yr'].run)):
    OBS_unforced_trend_pattern_correlation_30yr.append(stats.pearsonr(ACCESS_unforced_30yr_trend.isel(run=i).values.flatten(), Obs_ACCESS_unforced_30yr_trend['tas'].values.flatten())[0])
    
OBS_unforced_trend_pattern_correlation_60yr = []
for i in range(len(unforced_trend_da['60yr'].run)):
    OBS_unforced_trend_pattern_correlation_60yr.append(stats.pearsonr(ACCESS_unforced_60yr_trend.isel(run=i).values.flatten(), Obs_ACCESS_unforced_60yr_trend['tas'].values.flatten())[0])

In [ ]:
# Save to a text file
with open('obsUnforced_ACCESSunforced_pattern_correlations.txt', 'w') as file:
    file.write('10-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_10yr:
        file.write(f"{correlation}\n")

    file.write('\n30-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_30yr:
        file.write(f"{correlation}\n")

    file.write('\n60-year Trend Pattern Correlations:\n')
    for correlation in OBS_unforced_trend_pattern_correlation_60yr:
        file.write(f"{correlation}\n")

In [ ]:
OBS_unforced_trend_pattern_correlation_10yr

In [ ]:
client.close()
scluster.close()